In [2]:
import numpy as np
import matplotlib; matplotlib.use("TkAgg") #this line makes the animation run in the IDE. Comment if you only want to export it
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import colors
from PIL import Image
from matplotlib.patches import Circle


# Create a forest fire animation based on a simple cellular automaton model.
# The maths behind this code is described in the scipython blog article
# at https://scipython.com/blog/the-forest-fire-model/
# Christian Hill, January 2016.
# Updated January 2020.

# Displacements from a cell to its eight nearest neighbours
neighbourhood = ((-1,-1), (-1,0), (-1,1), (0,-1), (0, 1), (1,-1), (1,0), (1,1))
EMPTY, TREE, FIRE, WALL = 0, 1, 2, 3
# Colours for visualization: brown for EMPTY, dark green for TREE and orange
# for FIRE. Note that for the colormap to work, this list and the bounds list
# must be one larger than the number of different values in the array.
colors_list = [(0.2,0,0), (0,0.5,0), 'orange', 'white']
cmap = colors.ListedColormap(colors_list)
bounds = [0,1,2,3,4]
norm = colors.BoundaryNorm(bounds, cmap.N)


# load in the map
image_path = "C:\\Users\\owmat\\Downloads\\testroom.bmp"
image = Image.open(image_path)
image_array = np.array(image)

mapping = {0: 3, 1: 0}
mapped_array = np.vectorize(mapping.get)(image_array)

print(mapped_array)

def iterate(X):
    """Iterate the forest according to the forest-fire rules."""

    # The boundary of the forest is always empty, so only consider cells
    # indexed from 1 to nx-2, 1 to ny-2
    X1 = np.zeros((ny, nx))
    for ix in range(1,nx-1):
        for iy in range(1,ny-1):
            if X[iy,ix] == WALL:
                X1[iy,ix] = WALL
            if X[iy,ix] == EMPTY and np.random.random() <= p:
                X1[iy,ix] = TREE
            if X[iy,ix] == TREE:
                X1[iy,ix] = TREE
                for dx,dy in neighbourhood:
                    # The diagonally-adjacent trees are further away, so
                    # only catch fire with a reduced probability:
                    if abs(dx) == abs(dy) and np.random.random() < 0.573:
                        continue
                    if X[iy+dy,ix+dx] == FIRE:
                        X1[iy,ix] = FIRE
                        break
                else:
                    if np.random.random() <= f:
                        X1[iy,ix] = FIRE
    return X1

# The initial fraction of the forest occupied by trees.
forest_fraction = 0.3
wall_fraction = 0.3
# Probability of new tree growth per empty cell, and of lightning strike.
p, f = 0.02, 0.00005
# Forest size (number of cells in x and y directions).
nx, ny = 100, 100
# Initialize the forest grid.
X  = np.zeros((ny, nx))
X[1:ny-1, 1:nx-1] = np.random.randint(0, 2, size=(ny-2, nx-2))
X[1:ny-1, 1:nx-1] = np.random.random(size=(ny-2, nx-2)) < forest_fraction
#X[1:ny-1, 1:nx-1] = np.where(np.random.random(size=(ny-2, nx-2)) < wall_fraction, 3, 0)
#X[1:ny-1, 1:nx-1] = np.random.random(0, 3, size=(ny-2, nx-2)) < wall_fraction
print(X)

X = mapped_array




fig = plt.figure(figsize=(25/3, 6.25))
ax = fig.add_subplot(111)
ax.set_axis_off()
im = ax.imshow(X, cmap=cmap, norm=norm)#, interpolation='nearest')
#plt.show()
# Light strip config
light_y = ny - 20  # horizontal strip lower in the map
light_xs = list(range(10, nx - 10, 5))  # Light segments spaced
light_range = 4
light_scatter = ax.scatter(light_xs, [light_y]*len(light_xs), color='red', s=30)


# Check fire presence nearby
def check_safety(grid, x, y, r):
    x_min = max(0, x - r)
    x_max = min(grid.shape[1], x + r + 1)
    y_min = max(0, y - r)
    y_max = min(grid.shape[0], y + r + 1)
    area = grid[y_min:y_max, x_min:x_max]
    return not np.any(area == FIRE)

# === SPEAKER SETUP ===
speaker_positions = [(25, 25), (75, 25), (25, 85), (85, 85)]
speaker_radius = 15
speaker_circles = []

for (x, y) in speaker_positions:
    ax.plot(x, y, 'ko', markersize=5)  # Speaker center dot
    circle = Circle((x, y), speaker_radius, color='blue', alpha=0.2)
    ax.add_patch(circle)
    speaker_circles.append(circle)

# The animation function: called to produce a frame for each generation.
def animate(i):
    im.set_data(animate.X)
    animate.X = iterate(animate.X)

    # Update light strip color
    safe_status = [check_safety(animate.X, x, light_y, light_range) for x in light_xs]
    colors = ['cyan' if safe else 'red' for safe in safe_status]
    light_scatter.set_offsets(np.c_[light_xs, [light_y]*len(light_xs)])
    light_scatter.set_color(colors)

# Bind our grid to the identifier X in the animate function's namespace.
animate.X = X

# Interval between frames (ms).
interval = 100
anim = animation.FuncAnimation(fig, animate, interval=interval, frames=200)
#anim.save("forest_fire.mp4")
html_out = anim.to_jshtml()
with open('anim.html', "w") as tf:
    tf.write(html_out)
plt.show()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
